In [57]:
E = 210000000
v=0.3
t=0.025
w=3000
import numpy as np

In [58]:

nodes = [(0,0), (0.5,0), (0.5,0.25), (0,0.25)]
element_nodes=[(1,3,4),(1,2,3)] 
element_areas=[]
element_k = []
element_b = []
D = (E/(1-v**2)) * (np.array([1,v,0,v,1,0,0,0,(1-v)/2]).reshape((3,3)))
for element in element_nodes:
    x = []
    y = []
    for node in element:
        x.append(nodes[node-1][0])
        y.append(nodes[node-1][1])
    area_mat = np.array([[1,1,1],x,y])
    area = 0.5*np.linalg.det(area_mat)
    element_areas.append(area)
    
    b1 = y[1] - y[2]
    b2 = y[2] - y[0]
    b3 = y[0] - y[1]
    g1 = x[2] - x[1]
    g2 = x[0] - x[2]
    g3 = x[1] - x[0]
    
    B = np.array([[b1,0,b2,0,b3,0],[0,g1,0,g2,0,g3],[g1,b1,g2,b2,g3,b3]]) / (2*area)
    element_b.append(B)
    
    k = np.matmul(np.transpose(B),np.matmul(D,B))*area*t
    element_k.append(k)



In [59]:
K = np.zeros((len(nodes)*2,len(nodes)*2))
for node in range(len(element_nodes)):
    node_pos = []
    for i in element_nodes[node]:
        node_pos.append(i*2-2)
        node_pos.append(i*2-1)
    for i in range(len(node_pos)):
        K[node_pos[i]][node_pos] += np.transpose(element_k[node])[i]


In [60]:

U=np.zeros((len(nodes)*2,1))  
F=np.zeros((len(nodes)*2,1))  

F[2,0]=9.375
F[3,0]=0
F[4,0]=9.375
F[5,0]=0

Up=U[[2,3,4,5]]
Fp=F[[2,3,4,5]]
Kpp=K[[2,3,4,5]]
Kpp=Kpp[:,[2,3,4,5]]

Up=np.dot(np.linalg.inv(Kpp),Fp)
print("Nodal Displacements of [U2x U2y U3x U3y]' (in m)")
print(Up)

Nodal Displacements of [U2x U2y U3x U3y]' (in m)
[[7.11111747e-06]
 [1.11517786e-06]
 [6.53122498e-06]
 [4.46071143e-08]]


In [61]:

U[[2,3,4,5]]=Up

F=np.dot(K,U)
print("Force Vector [F1x0 F1y F2x F2y F3x F3y F4x F4y](in kN)")
print(F)

Force Vector [F1x0 F1y F2x F2y F3x F3y F4x F4y](in kN)
[[-9.37500000e+00]
 [-5.62950360e+00]
 [ 9.37500000e+00]
 [ 1.77635684e-15]
 [ 9.37500000e+00]
 [-2.66453526e-15]
 [-9.37500000e+00]
 [ 5.62950360e+00]]


In [62]:
u=[]
for node in element_nodes:
    u1 = np.asarray([U[node[0]*2-2],U[node[0]*2-1],U[node[1]*2-2],U[node[1]*2-1],U[node[2]*2-2],U[node[2]*2-1]])
    u.append(u1)


In [63]:
elem_stress_sigma=[]
for i in range(len(element_nodes)):
    temp = np.dot(D,np.dot(element_b[i],u[i]))
    elem_stress_sigma+=[temp]
elem_stress_sigma

[array([[3014.41152922],
        [ 904.32345877],
        [   7.20576461]]),
 array([[2985.58847078],
        [  -3.60288231],
        [  -7.20576461]])]

In [64]:
def principal_stress(sigma):
    R = (sigma[0] + sigma[1])/2
    Q = ((sigma[0] - sigma[1])/2)**2 + sigma[2]*sigma[2]
    M = 2*sigma[2]/(sigma[0] - sigma[1])
    s1 = R + np.sqrt(Q)
    s2 = R - np.sqrt(Q)
    theta = (np.arctan(M)/2)*180/np.pi
    return np.asarray([s1 , s2 , theta])

In [65]:
prin_stress = []
for sigma in elem_stress_sigma:
    prin_stress.append(principal_stress(sigma))
prin_stress

[array([[3.01443614e+03],
        [9.04298852e+02],
        [1.95656990e-01]]),
 array([[ 2.98560584e+03],
        [-3.62025247e+00],
        [-1.38116518e-01]])]